In [1]:
from data_provider import DataModel
dm = DataModel()
dm.read_from_csvs("../../xcels", ["master0.csv", "master1.csv"])
df = dm.get(dm.TA_SYMBOLS[0], "1395-01-01", "1398-01-01")

2020-02-13 11:26:03.666 | DEBUG    | data_provider:get:172 - symbol is not scaled: خپارس
2020-02-13 11:26:03.678 | DEBUG    | data_provider:adjust_and_log:57 - start adjust and log for خپارس
2020-02-13 11:26:03.680 | DEBUG    | data_provider:adjust_and_log:58 - calculating scale
2020-02-13 11:26:03.729 | DEBUG    | data_provider:adjust_and_log:60 - adding adjust
2020-02-13 11:26:03.732 | DEBUG    | data_provider:add_adjust:28 - new part len is 3459, shape: (3459,)
2020-02-13 11:26:03.760 | DEBUG    | data_provider:add_adjust:38 - found adj date: 2009-06-08 00:00:00
2020-02-13 11:26:03.772 | DEBUG    | data_provider:add_adjust:38 - found adj date: 2010-07-05 00:00:00
2020-02-13 11:26:03.782 | DEBUG    | data_provider:add_adjust:38 - found adj date: 2011-07-05 00:00:00
2020-02-13 11:26:03.791 | DEBUG    | data_provider:add_adjust:38 - found adj date: 2012-06-23 00:00:00
2020-02-13 11:26:03.799 | DEBUG    | data_provider:add_adjust:38 - found adj date: 2014-05-26 00:00:00
2020-02-13 11:26

In [ ]:
dm.TA_SYMBOLS[0]

In [5]:
# %%writefile ta_module.py

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from datetime import datetime
from math import pi
import math
# import trade_model as td
import matplotlib.pyplot as plt
from data_provider import DataModel


accepted_error=0.01



def find_optimum_points(data : list, ismax : bool = True, days = 3):# if input = dataframe it takes 50 times more time
    last = -1000
    x, y = [], []
    if ismax:
        for j in range(len(data)): 
            if (j > days) and (j + days < len(data)):
                if (data[j] >= max(data[j - days: j + days]) and last < j - days):
                    last = j
                    x.append(j)
                    y.append(data[j])
    if not(ismax):
        for j in range(len(data)): 
            if (j > days) and (j + days < len(data)):
                if (data[j]<=min(data[j - days: j + days]) and last < j - days):
                    last = j
                    x.append(j)
                    y.append(data[j])
    df=pd.DataFrame({"x":x ,"y":y})
    return df




#########################################################################
#########################################################################
#########################################################################
#########################################################################


def find_trendlines(df, minimum_touch = 3, mode = "res"): # mode can be either "res" or "sup"
    if mode == "res":
        final_points = []
        points = find_optimum_points(df.log_adj_close.tolist())
        for p1 in range(len(points)):
            for p2 in range(p1+1, len(points)):
                a = (points.y[p2] - points.y[p1]) / (points.x[p2] - points.x[p1])
                b = points.y[p1] - (a * points.x[p1])
                if len(points[p1:p2][points.x*a+b < points.y + math.log10(1 - accepted_error)]) == 0 :

        #                BEFORE : if(x_*a+b>=math.log10((10**y_)-((10**y_)*accepted_error))) and (x_*a+b<=math.log10((10**y_)+((10**y_)*accepted_error))):
                    if len(points[p1:p2][np.logical_and(points.x * a + b >= points.y + math.log10(1 - accepted_error) ,
                                                 points.x * a + b <= points.y + math.log10(1 + accepted_error))])>= minimum_touch:

                        final_points.append([points.x[p1], points.x[p2]])
        return final_points
    
    if mode == "sup":
        final_points = []
        points = find_optimum_points(df.log_adj_close.tolist(), ismax=False)
        for p1 in range(len(points)):
            for p2 in range(p1+1, len(points)):
                a = (points.y[p2] - points.y[p1]) / (points.x[p2] - points.x[p1])
                b = points.y[p1] - (a * points.x[p1])
                if len(points[p1:p2][points.x*a+b > points.y + math.log10(1 + accepted_error)]) == 0 :

    #                BEFORE : if(x_*a+b>=math.log10((10**y_)-((10**y_)*accepted_error))) and (x_*a+b<=math.log10((10**y_)+((10**y_)*accepted_error))):
                    if len(points[p1:p2][np.logical_and(points.x * a + b >= points.y + math.log10(1 - accepted_error) ,
                                                 points.x * a + b <= points.y + math.log10(1 + accepted_error))])>= minimum_touch:
    #                     if :   # delete repetitious lines
                        final_points.append([points.x[p1], points.x[p2], a, b])
        return final_points
   

In [60]:
yy = df.log_adj_close.tolist()
# points_min = find_optimum_points(df.log_adj_close.tolist(), ismax=False)
# points_max = find_optimum_points(df.log_adj_close.tolist())
# final_points_sup = find_trendlines(df, mode = "sup")
# final_points_res = find_trendlines(df, mode = "res")
plt.figure(figsize = (15, 7))
plt.plot(np.arange(len(df)), df.log_adj_close)
for i in final_points_sup:
    plt.plot([i[0],i[1]], [yy[i[0]], yy[i[1]]])
# for i in final_points_res:
#     plt.plot([i[0],i[1]], [yy[i[0]], yy[i[1]]])
    
x = np.array(points_min.x.tolist())
y = np.array(points_min.y.tolist())
xx = np.array(max_in_touch.x.tolist())
yy = np.array(max_in_touch.y.tolist())
plt.scatter(xx, yy)
plt.scatter(x, y)
# plt.scatter(xx, yy + math.log10(1 + accepted_error))
# plt.scatter(xx, yy + math.log10(1 - accepted_error))


In [19]:
len(final_points_sup)

83

In [22]:
%%time
suplines = pd.DataFrame(find_trendlines(df, mode = "sup"), columns=["start", "stop", "coef", "intercept"])

/home/nimac/.local/lib/python3.6/site-packages/ipykernel_launcher.py:71: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.

/home/nimac/.local/lib/python3.6/site-packages/ipykernel_launcher.py:75: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



CPU times: user 7.34 s, sys: 19.8 ms, total: 7.36 s
Wall time: 7.35 s


In [56]:
max_in_touch=pd.DataFrame([])
for ind,r in suplines.iterrows():
    touch = points_max[np.logical_and( ((points_max.x * r.coef) + r.intercept)> points_max.y + math.log10(1 - accepted_error),
                                              ((points_max.x * r.coef) + r.intercept)< points_max.y + math.log10(1 + accepted_error))]
    touch = touch[touch.x > r.stop]
    if(len(touch) != 0):
        max_in_touch = max_in_touch.append(touch.iloc[0])
#         print("hiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiiii")

In [59]:
max_in_touch

,x,y
21,217.0,2.955688
21,217.0,2.955688
30,299.0,2.979548
29,292.0,2.973128
29,292.0,2.973128
34,331.0,2.967080
29,292.0,2.973128
44,424.0,2.935507
43,417.0,2.923244
51,480.0,2.871573
